In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import argparse
import matplotlib.cm as cm
import os
#import torch

from mpl_toolkits.mplot3d import Axes3D



def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--scenario", type=str, default=f'dummy')   #set secnario to "dummy" / "efficiency" / "comfort"
    
    parser.add_argument("--cooling_rate", type=int, default=1)      #not currently implemented default 1
    parser.add_argument("--heating_rate", type=float, default=0.663)

    parser.add_argument("--comfort_temp", type=int, default=22)
    parser.add_argument("--efficiency_temp", type=int, default=20) 
    parser.add_argument("--resting_temp", type=int, default=18)

    parser.add_argument("--comfort_tw", type=int, default=15)
    parser.add_argument("--efficiency_tw", type=int, default=30)

    return parser.parse_args("")
args = parse_args()


In [39]:
#####Physics
#Annahmen:
#Volumen = Bodenfläche * Höhe = 15 m² * 2,5 m = 37,5 m³
#Masse = Volumen * Dichte = 37,5 m³ * 1,2 kg/m³ = 45 kg     (Dichte von Luft = 1.2 kg/m³)
#Leistung der Heizung = 0,5KW
#Aufheizungsrate = Leistung / (Masse * spezifische Wärmekapazität) = 0,0110558 C/s = 500 J/s / (45 kg * 1005 J/(kg·°C))
#0,66335 C/m ≈ 0,663 C/m
#
#Assumed Heating Rate of 0,663 C/m

In [40]:
def create_timeline(data):
    print('Creating Timeline for algorithm application...')
    df = pd.DataFrame(data)
    df.set_index('minute_idx', inplace=True)

    # Create a new DataFrame with the desired index range
    new_index = range(df.index.min(), df.index.max()+1)
    new_df = df.reindex(new_index)

    # Forward-fill missing values in the DataFrame
    new_df.ffill(inplace=True)

    new_df.reset_index(inplace=True)
    return new_df

def update_in_usage(df, consecutive_rows):
    print('Updating room usage during every point in time...')

    count = 0
    for i in range(len(df) - consecutive_rows + 1):
        if all(df.loc[i+j, 'PersonenAnzahl'] > 0 for j in range(consecutive_rows)):
            df.loc[i:i+consecutive_rows-1, 'in_usage'] = True
    return df

def update_temp(df, target_temp):
    print('Setting temperature during in usage periods to target temperature...')
    df.loc[df['in_usage'], 'temp'] = target_temp
    df.loc[df['in_usage'], 'heating'] = True
    return df


def update_temperature(df, target_temp):
    print('Beginning heating and cooling simulation...')
    length_df = len(df)
    old_usage_start = 0
    old_usage_end = 0
    complete = False
         
    c = 1
    while complete == False:
        
        print(f'-- Simulation iteration {c} --')
        df, old_usage_start, old_usage_end, complete = temperature_change(df, old_usage_start, old_usage_end, complete, target_temp)
        c+=1

    return df


def temperature_change(df, old_usage_start, old_usage_end, complete, target_temp):

    heating_rate = args.heating_rate

    current_line = 0
    usage_start = old_usage_start
    usage_end = old_usage_end
    for index, row in df.iterrows():
        if old_usage_end < index:
            if row['temp'] == target_temp:
                usage_start = index
                previous_index = index - 1
                break
    i = 0

    ###AUFHEIZEN
    for index, row in df.iterrows():
        if old_usage_end < index:
            for index, row in df.iterrows():
                if old_usage_end < index:
                    if usage_start - (i+1) == index:
                        #falls (temperatur bei i - 1) kleiner ist als (temperatur bei i) - heating_rate
                        if df.at[usage_start - (i+1), 'temp'] <= (df.at[usage_start - i, 'temp'] - heating_rate):
                            df.at[usage_start - (i+1), 'temp'] = df.at[usage_start - i, 'temp'] - heating_rate
                            df.at[usage_start - (i+1), 'heating'] = True
                            i+=1
                        else: 
                            break
    j = 0
    index = 0

    

    ###ABKÜHLEN
    block_done = False
    usage_end_first_time = False
    for index, row in df.iterrows():
        #Check that usage block hasnt been done already
        if old_usage_end < index:
            #Check that past beginning of new usage block
            if usage_start <= index:
                #Check that at first value after usage block
                #if row['temp'] != target_temp:
                if row['in_usage']!= True:
                    if usage_end_first_time == False:
                        usage_end = index - j - 1     
                        usage_end_first_time = True
                    #Sind wir noch im dataframe
                    if len(df) >= index - 1:
                        #
                        if df.at[index, 'temp'] >= args.resting_temp:
                            if (target_temp - j - 1) >= args.resting_temp:
                                df.at[index, 'temp'] = (target_temp - j - 1)
                                j+=1
                elif (usage_end <= index) and (row['temp'] == target_temp) and old_usage_end != usage_end:
                    return df, usage_start, usage_end, complete
        #Sind wir am Ende des DataFrame
        if index == (len(df) - 1):
            complete = True
            return df, usage_start, usage_end, complete

def prototype_calc_energy_usage(df):
    print('Calculating estimated energy usage...')

    minutes = df['heating'].value_counts()
    #_, minutes = df['heating'].value_counts()
    leistung = 1 # Kilo watt pro stunde
    return (minutes / 60 * leistung)

def process_RGB_values(df, min_temperature, max_temperature):
    print('Generating RGB values from temperatures...')
    room_df = df
    normalized_temperatures = (room_df['temp'] - min_temperature) / (max_temperature - min_temperature)
    cmap = cm.get_cmap('coolwarm')
    rgb_values = normalized_temperatures.apply(lambda x: cmap(x)[:3])
    room_df['rgb_value'] = rgb_values
    room_df['rgb_value'] = room_df['rgb_value'].apply(lambda rgb: tuple(int(val * 255) for val in rgb))
    room_df = room_df.reset_index(drop=True)

    for index, row in room_df.iterrows():
        x = row['rgb_value']
        x = str(x)
        x = x.replace(',' , ';')
        room_df.at[ index, 'rgb_value'] = str(x)

    return room_df

def scenario_dummy(file_name):
    print('Generating dummy heating scenario...')
    df = pd.read_csv(f'data/{file_name}')
    df = create_timeline(df)
    df = df.drop('temp', axis=1)
    df['temp'] = args.resting_temp
    df = df.drop('rgb_value', axis=1)
    # Add the 'in_usage' column and set the initial value to False
    df['in_usage'] = True  
    # Add the 'heating' column and set the initial value to False
    df['heating'] = True
    #df.to_csv(f'4C_05_{args.scenario}.csv')
    save_df(df, file_name)
    kilowattstunden = prototype_calc_energy_usage(df)
    print(f'Der Energieverbrauch im verwendeten Scenario "{args.scenario}" betrug {kilowattstunden} KWH.')
    return df

def next_person_enters(df):
    print('Processing information about next time of arrival...')
    df['next_person_enters'] = False
    current_nr_people = 0

    for index, row in df.iterrows():
        if int(row['PersonenAnzahl']) > int(current_nr_people):
            df.at[ index, 'next_person_enters'] = True
            current_nr_people = row['PersonenAnzahl']
        elif row['PersonenAnzahl'] <= current_nr_people:
            current_nr_people = row['PersonenAnzahl']


    previous_arrival = 0
    minute_person_arrived = 0
    for index, row in df.iterrows():
        if (row['next_person_enters'] == True and row['in_usage'] == True):
            minute_person_arrived = row['minute_idx']
            for index, row in df.iterrows(): 
                if index >= previous_arrival:
                    df.at[index - 1, 'next_person_enters'] = minute_person_arrived
                    if index == minute_person_arrived:
                        previous_arrival = minute_person_arrived
                        break

    
    first_person_this_week = df.at[0, 'next_person_enters']
    #for index, row in df.iterrows():
    for index, row in df.iterrows():
        if df.at[index, 'next_person_enters'] == False:
            df.at[index, 'next_person_enters'] = first_person_this_week
        if df.at[index, 'next_person_enters'] == True:
            df.at[index, 'next_person_enters'] = first_person_this_week


    return df


##functionality already present in timeline code
def add_minute_zero(df):
    ##insert function
    print('Adding minute 0 to the dataframe...')

    if df.at[0, 'minute_idx'] != 0:
        print('test')
        new_entry = {'minute_idx': 0, 'PersonenAnzahl': 0.0, 'temp': args.resting_temp, 'rgb_value': (0, 0, 0)}

        # Create a new DataFrame with the new entry
        new_df = pd.DataFrame([new_entry])
        # Concatenate the new row with the existing DataFrame
        df = pd.concat([new_df, df]).reset_index(drop=True)

    elif df.at[0, 'minute_idx'] == 0:
        print('Minute 0 already present...')
        
    return df

def save_df(df, file_name):
    df.to_csv(f'final_dataframes/{args.scenario}/{args.scenario}_{file_name}')


def get_files_in_directory(directory_path):
    try:
        # Get a list of all files in the directory
        file_list = os.listdir(directory_path)
        return file_list
    except OSError:
        print(f"Error: Unable to read files from the directory '{directory_path}'")
        return []

def heating_algo(file_name):
    df = pd.read_csv(f'data/{file_name}')
    print('Initiating heating algorithm...')
    #Settings
    if args.scenario == 'comfort':
        target_temp = args.comfort_temp
        time_window = args.comfort_tw
    elif args.scenario == 'efficiency':
        target_temp = args.efficiency_temp
        time_window = args.efficiency_tw
    else:
        print('Scenario not found...')


    df = add_minute_zero(df)
    df = create_timeline(df)
    df = df.drop('temp', axis=1)
    df['temp'] = args.resting_temp
    df = df.drop('rgb_value', axis=1)
    # Add the 'in_usage' column and set the initial value to False
    df['in_usage'] = False  
    # Add the 'heating' column and set the initial value to False
    df['heating'] = False
    df= update_in_usage(df, time_window)
    df = update_temp(df, target_temp)
    df = update_temperature(df, target_temp)
    df = process_RGB_values(df, args.resting_temp, target_temp)
    df = next_person_enters(df)
    #df.to_csv(f'4C_05_light_scenario:{args.scenario}_hr:{args.heating_rate}_tw:{time_window}_tt:{target_temp}_rt:{args.resting_temp}.csv')

    save_df(df, file_name)

    kilowattstunden = prototype_calc_energy_usage(df)
    print(f'Der Energieverbrauch im verwendeten Scenario "{args.scenario}" betrug {kilowattstunden} KWH.')
    print('-- Success --')
    return df

def generate_all_datasets():
    directory_path = r'C:\Users\lukas\Desktop\Seminar ARVR\vscode\data'
    # Call the function to get the file names in the directory
    files = get_files_in_directory(directory_path)
    for file in files:
        heating_algo(file)

def generate_dummy_datasets():
    directory_path = r'C:\Users\lukas\Desktop\Seminar ARVR\vscode\data'
    # Call the function to get the file names in the directory
    files = get_files_in_directory(directory_path)
    for file in files:
        scenario_dummy(file)



In [ ]:
##HOW TO USE HEATING ALGO:
file = r'C:\Users\lukas\Desktop\Seminar ARVR\vscode\data\4C-01.1.csv'
heating_algo(file)

_______________________________________________________________________________________

Old Code
----------------------

In [41]:
generate_dummy_datasets()

Generating dummy heating scenario...
Creating Timeline for algorithm application...
Calculating estimated energy usage...
Der Energieverbrauch im verwendeten Scenario "dummy" betrug True    35.1
Name: heating, dtype: float64 KWH.
Generating dummy heating scenario...
Creating Timeline for algorithm application...
Calculating estimated energy usage...
Der Energieverbrauch im verwendeten Scenario "dummy" betrug True    1.783333
Name: heating, dtype: float64 KWH.
Generating dummy heating scenario...
Creating Timeline for algorithm application...
Calculating estimated energy usage...
Der Energieverbrauch im verwendeten Scenario "dummy" betrug True    16.233333
Name: heating, dtype: float64 KWH.
Generating dummy heating scenario...
Creating Timeline for algorithm application...
Calculating estimated energy usage...
Der Energieverbrauch im verwendeten Scenario "dummy" betrug True    48.283333
Name: heating, dtype: float64 KWH.
Generating dummy heating scenario...
Creating Timeline for algorit

In [ ]:
time_window = args.comfort_tw

df = pd.read_csv("data/4C-05.csv")
df = add_minute_zero(df)
df = create_timeline(df)
df = df.drop('temp', axis=1)
df['temp'] = args.resting_temp
df = df.drop('rgb_value', axis=1)
# Add the 'in_usage' column and set the initial value to False
df['in_usage'] = False  
# Add the 'heating' column and set the initial value to False
df['heating'] = False
df = update_in_usage(df, time_window)
df = update_temp(df, 22)
#df = update_temperature(df, 22)
df = process_RGB_values(df, args.resting_temp, 22)
#df = next_person_enters(df)
#data = heating_algo(data)
df.head(10)


Adding minute 0 to the dataframe...
test
Creating Timeline for algorithm application...
Updating room usage during every point in time...
Setting temperature during in usage periods to target temperature...
Generating RGB values from temperatures...


C:\Users\lukas\AppData\Local\Temp\ipykernel_2904\4018629484.py:141: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('coolwarm')


,minute_idx,PersonenAnzahl,temp,in_usage,heating,rgb_value
0,0,0.0,18,False,False,(58; 76; 192)
1,1,0.0,18,False,False,(58; 76; 192)
2,2,0.0,18,False,False,(58; 76; 192)
3,3,0.0,18,False,False,(58; 76; 192)
4,4,0.0,18,False,False,(58; 76; 192)
5,5,0.0,18,False,False,(58; 76; 192)
6,6,0.0,18,False,False,(58; 76; 192)
7,7,0.0,18,False,False,(58; 76; 192)
8,8,0.0,18,False,False,(58; 76; 192)
9,9,0.0,18,False,False,(58; 76; 192)


In [100]:
##TODO AVOID ALGO SEARCHING AFTER FINISHED UPDATING ONE BLOCK
data = pd.read_csv("zeitreihen/4C-05_light.csv")
df = heating_algo(data)


Initiating Heating Algorithm...
Creating Timeline for algorithm application...
Updating whether room is in usage during given point in time...
Setting temperature during in usage periods to target temperature...
Beginning heating and cooling simulation...
-- Simulation iteration 1 --
-- Simulation iteration 2 --
-- Simulation iteration 3 --
-- Simulation iteration 4 --
-- Simulation iteration 5 --
-- Simulation iteration 6 --
Generating RGB values from temperatures...
Processing information about next time of arrival...


C:\Users\lukas\AppData\Local\Temp\ipykernel_532\102142437.py:142: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('coolwarm')


Calculating estimated energy usage...
Der Energieverbrauch im verwendeten Scenario "comfort" betrug 7.1 KWH.


In [90]:
df = data.copy()



df = next_person_enters(df)

df.head(73)

,Unnamed: 0,minute_idx,PersonenAnzahl,temp,in_usage,heating,rgb_value,next_person_enters
0,0.0,1.0,0.0,18.00,False,False,(58; 76; 192),71
1,1.0,2.0,0.0,18.00,False,False,(58; 76; 192),71
2,2.0,3.0,0.0,18.00,False,False,(58; 76; 192),71
3,3.0,4.0,0.0,18.00,False,False,(58; 76; 192),71
4,4.0,5.0,0.0,18.00,False,False,(58; 76; 192),71
...,...,...,...,...,...,...,...,...
68,68.0,69.0,0.0,21.34,False,True,(230; 116; 90),71
69,69.0,70.0,0.0,21.67,False,True,(209; 73; 62),71
70,70.0,71.0,1.0,22.00,True,True,(179; 3; 38),154
71,71.0,72.0,1.0,22.00,True,True,(179; 3; 38),154


In [44]:
df = df_bu.copy()




df.head()
print(df['rgb_value'][0])

print(df['rgb_value'][0])

(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(58, 76, 192)
(60, 79, 195)
(86, 115, 224)
(113, 148, 244)
(142, 177, 253)
(172, 200, 252)
(199, 214, 240)
(222, 219, 218)
(239, 205, 187)
(246, 182, 154)
(243, 152, 121)
(230, 116, 90)
(209, 73, 62)
(1

In [ ]:
def scenario_dummy(data):
    df = create_timeline(data)
    df = df.drop('temp', axis=1)
    df['temp'] = args.resting_temp
    df = df.drop('rgb_value', axis=1)
    # Add the 'in_usage' column and set the initial value to False
    df['in_usage'] = True  
    # Add the 'heating' column and set the initial value to False
    df['heating'] = True
    df.to_csv(f'4C_05_light_scenario_{args.scenario}.csv')
    return df

In [23]:
#Scnario Dummy Heizen

data = pd.read_csv("zeitreihen/4C-05_light.csv")
df = scenario_dummy(data)
kilowattstunden = prototype_calc_energy_usage(df)
print(f'Der Energieverbrauch im verwendeten Scenario "{args.scenario}" betrug {kilowattstunden} KWH.')
df

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
#Output Comfort Scenario
#Iteration: 1 Old Usage Start: 0
#Iteration: 2 Old Usage Start: 71
#Iteration: 3 Old Usage Start: 154
#Iteration: 4 Old Usage Start: 310
#Iteration: 5 Old Usage Start: 684
#Iteration: 6 Old Usage Start: 769
#Der Energieverbrauch im verwendeten Scenario "comfort" betrug 7.1 KWH.

In [ ]:
#Output Efficiency Scenario
#Iteration: 1 Old Usage Start: 0
#Iteration: 2 Old Usage Start: 71
#Iteration: 3 Old Usage Start: 154
#Iteration: 4 Old Usage Start: 310
#Iteration: 5 Old Usage Start: 684
#Iteration: 6 Old Usage Start: 769
#Der Energieverbrauch im verwendeten Scenario "efficiency" betrug 6.5 KWH.

------------------------------------------------------------

In [ ]:
data = {
    "minute_idx": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16 ,17, 18, 19, 20, 21, 22, 23, 24, 25],
    "PersonenAnzahl": [1, 2, 1, 0, 5, 1, 3, 0, 1, 1, 5, 0, 1, 1, 1, 0, 5, 4, 5, 0, 1, 1, 2, 4, 5],
    "temp": [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]
}
data = pd.DataFrame(data)

In [ ]:
data = {
    "minute_idx": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16 ,17, 18, 19, 20, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16 ,17, 18, 19, 20, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16 ,17, 18, 19, 20],
    "PersonenAnzahl": [0, 0, 0, 4, 5, 1, 3, 2, 0, 0, 5, 0, 0, 0, 0, 0, 5, 4, 5, 0, 0, 0, 0, 4, 5, 1, 3, 2, 0, 0, 5, 0, 0, 0, 0, 0, 5, 4, 5, 0, 0, 0, 0, 4, 5, 1, 3, 2, 0, 0, 5, 0, 0, 0, 0, 0, 5, 4, 5, 0],
    "temp": [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]
}

df_base = pd.DataFrame(data)
df_base['in_usage'] = False  # Add the 'in_usage' column and set the initial value to False
df_base['heating'] = False
df_base = update_in_usage(df_base, 3)
df_base = update_temp(df_base)


In [ ]:
import pandas as pd

def heat_room_v1(df):
    df["heating"] = False
    df["temp"] = df["temp"].astype(float)

    start_heating = None
    for i, row in df.iterrows():
        if row["PersonenAnzahl"] > 0 and row["temp"] < 22:
            start_heating = i
            break

    if start_heating is not None:
        df.loc[start_heating, "heating"] = True

        for i, row in df.iterrows():
            if i < start_heating:
                continue

            elapsed_minutes = row["minute_idx"] - df.loc[start_heating, "minute_idx"]

            if row["PersonenAnzahl"] > 0:
                df.loc[i, "temp"] = min(22, df.loc[start_heating, "temp"] + elapsed_minutes * 1.33)
            else:
                df.loc[i, "temp"] = max(18, df.loc[start_heating, "temp"] - elapsed_minutes)

            if row["temp"] >= 22:
                break

    return df

In [ ]:
import pandas as pd

def heizungs_algorithmus(dataframe):
    # Kopie des Eingabe-DataFrames erstellen, um den aktualisierten DataFrame zu erstellen
    updated_dataframe = dataframe.copy()
    
    # Spalte "heating" hinzufügen und mit False initialisieren
    updated_dataframe["heating"] = False
    
    # Durch den DataFrame iterieren
    for index, row in updated_dataframe.iterrows():
        # Wenn "PersonenAnzahl" größer als 0 ist
        if row["PersonenAnzahl"] > 0:
            # Temperatur auf 22°C setzen
            updated_dataframe.at[index, "temp"] = 22.0
            
            # Startpunkt für den Heizprozess ermitteln
            start_index = index - 15 if index >= 15 else 0
            start_temp = updated_dataframe.at[start_index, "temp"]
            
            # Heizprozess durchführen
            for i in range(start_index, index):
                # Wenn die Temperatur noch nicht 22°C erreicht hat
                if updated_dataframe.at[i, "temp"] < 22.0:
                    # Temperatur um 1,33°C pro Minute erhöhen
                    updated_dataframe.at[i, "temp"] += 1.33
                    # "heating" auf True setzen
                    updated_dataframe.at[i, "heating"] = True
                # Temperatur kann nicht über 22°C steigen
                updated_dataframe.at[i, "temp"] = min(updated_dataframe.at[i, "temp"], 22.0)
            
            # "heating" auf True setzen für den aktuellen Index
            updated_dataframe.at[index, "heating"] = True
        
        # Wenn "PersonenAnzahl" gleich 0 ist
        else:
            # Temperatur um 1°C pro Minute reduzieren
            updated_dataframe.at[index, "temp"] -= 1.0
            # Temperatur kann nicht unter 18°C fallen
            updated_dataframe.at[index, "temp"] = max(updated_dataframe.at[index, "temp"], 18.0)
    
    return updated_dataframe


In [ ]:
import pandas as pd

def update_in_usage(df):
    #df['in_usage'] = False  # Füge die 'in_usage'-Spalte hinzu und setze den initialen Wert auf 'False'

    
    for i in range(len(df) - 2):
        if df.loc[i, 'PersonenAnzahl'] > 0 and df.loc[i+1, 'PersonenAnzahl'] > 0 and df.loc[i+2, 'PersonenAnzahl'] > 0:
            df.loc[i:i+2, 'in_usage'] = True

    return df

In [ ]:

# Beispiel DataFrame
data = {
    "minute_idx": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16 ,17, 18, 19, 20],
    "PersonenAnzahl": [0, 0, 2, 4, 3, 1, 3, 2, 1, 0, 0, 0, 2, 4, 0, 1, 0, 0, 0, 0],
    "temp": [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]
}

df_base = pd.DataFrame(data)

# Funktion aufrufen
#result = heizungs_algorithmus(raum_4C_05)

# Ergebnis anzeigen
#print(result)

df = df_base.copy()


comfort_temp = 22

#df_base['temp'] = df_base['temp'].where(~(df_base['PersonenAnzahl'].rolling(window=1, center=True).sum() > 0), comfort_temp)
#df_base['temp'] = df_base.groupby((df_base['PersonenAnzahl'] > 0).cumsum())['temp'].transform(lambda x: x.mask(x.gt(0), 22))
#df_base.loc[df_base['PersonenAnzahl'].rolling(window=3, center=True).sum() > 0, 'temp'] = 22

df_base['in_usage'] = False




df_base = update_in_usage_22(df_base, 3)

df_base

NameError: name 'update_in_usage_22' is not defined